In [2]:
from selenium import webdriver
import pyttsx3 as p
from object_detection import *
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import time
import numpy as np
import cv2 
import pyautogui


class object_detection():
    def __init__(self):
        pass
        
    def get_info(self):
        
        WORKSPACE_PATH = 'flask\RealTimeObjectDetection-main\Tensorflow\workspace'
        SCRIPTS_PATH = 'flask\RealTimeObjectDetection-main\Tensorflow\scripts'
        APIMODEL_PATH = 'flask\RealTimeObjectDetection-main\Tensorflow\models'
        ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
        IMAGE_PATH = WORKSPACE_PATH+'/images'
        MODEL_PATH = WORKSPACE_PATH+'/models'
        PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
        CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
        CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'



        #####3#
        labels = [{'name':'play_pause', 'id':1}, {'name':'previous', 'id':2},{'name':'next', 'id':3}]

        with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
            for label in labels:
                f.write('item { \n')
                f.write('\tname:\'{}\'\n'.format(label['name']))
                f.write('\tid:{}\n'.format(label['id']))
                f.write('}\n')

        ####################################33        
        !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
        !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}


        #########################################33

        !cd flask\RealTimeObjectDetection-main\Tensorflow && git clone https://github.com/tensorflow/models

        ################################################

        CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

        !mkdir {'flask\RealTimeObjectDetection-main\Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
        #!copy {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

        CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

        config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
        ################################################################################33
        pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
        with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
            proto_str = f.read()                                                                                                                                                                                                                                          
            text_format.Merge(proto_str, pipeline_config)  
        #############################################################################################3
        pipeline_config.model.ssd.num_classes = 3
        pipeline_config.train_config.batch_size = 4
        pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
        pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
        pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
        pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
        pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
        pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

        ####################################################################


        config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
        with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
            f.write(config_text) 
        
    
        configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
        detection_model = model_builder.build(model_config=configs['model'], is_training=False)
        

        # Restore checkpoint
        ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
        ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-8')).expect_partial()

        @tf.function
        def detect_fn(image):
            image, shapes = detection_model.preprocess(image)
            prediction_dict = detection_model.predict(image, shapes)
            detections = detection_model.postprocess(prediction_dict, shapes)
            return detections
        category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
        cap = cv2.VideoCapture(0)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        while True: 
            ret, frame = cap.read()
            image_np = np.array(frame)

            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)

            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                          for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()

            viz_utils.visualize_boxes_and_labels_on_image_array(
                        image_np_with_detections,
                        detections['detection_boxes'],
                        detections['detection_classes']+label_id_offset,
                        detections['detection_scores'],
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw=1,
                        min_score_thresh=.5,
                        agnostic_mode=False)
            cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
            if(detections['detection_scores'][0]>0.9):
                return detections['detection_classes'][0]





class play_yt():
    def __init__(self):
        self.driver=webdriver.Chrome(executable_path=r'D:/hack/chromedriver.exe')
    def get_info(self,query):
        self.query=query
        self.driver.get(url="https://www.youtube.com/results?search_query="+query)
        
        # enter=self.driver.find_element_by_xpath('//*[@id="img"]')
        enter=self.driver.find_element_by_xpath('//*[@id="img"]')
        enter.click()
       
        engine=p.init()
        engine.say('playing'+query)
        engine.runAndWait()
#         info=self.driver.find_element_by_xpath('//*[@id="movie_player"]/div[32]/div[2]/div[1]/div[1]/span[1]')
#         readable_text=info.text
       
        while(True):
            s_info=self.driver.find_elements_by_class_name('ytp-time-current')
            for value in s_info:
                start=value.text
            
            e_info=self.driver.find_element_by_class_name('ytp-time-duration')
#             for value in e_info:
            end=e_info.text
            print(start,end)
            if(start==end):
                engine.say('gesture detection is deactivated')
                engine.runAndWait()
                break;
            else:
#                 try:
                engine.say('gesture detection has been activated')
                engine.runAndWait()
                obj=object_detection()
                cls=obj.get_info()
                
                if(cls==0):
                    pyautogui.press("k")
                elif(cls==1):
                    pyautogui.keyDown("shift")

                    # Presses the tab key once
                    pyautogui.press("n")

                    # Lets go of the alt key
                    pyautogui.keyUp("shift")
                else:
                    try:
                        enter=self.driver.find_element_by_xpath('//*[@id="movie_player"]/div[26]/div[2]/div[1]/a[1]')
                        enter.click()
                    except:
                        engine.say('there is not a playlist')
                        engine.runAndWait()



0:00 3:13
Successfully created the TFRecord file: flask\RealTimeObjectDetection-main\Tensorflow\workspace/annotations/train.record
Successfully created the TFRecord file: flask\RealTimeObjectDetection-main\Tensorflow\workspace/annotations/test.record


'git' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file flask\RealTimeObjectDetection-main\Tensorflow\workspace\models\my_ssd_mobnet already exists.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set